<a href="https://colab.research.google.com/github/jiin124/Dacon/blob/main/%EC%87%BC%ED%95%91%EB%AA%B0_%EC%A7%80%EC%A0%90%EB%B3%84_%EB%A7%A4%EC%B6%9C%EC%95%A1_%EC%98%88%EC%B8%A1/%EB%8D%B0%EC%9D%B4%EC%BD%98_Basic_%EC%87%BC%ED%95%91%EB%AA%B0_%EC%A7%80%EC%A0%90%EB%B3%84_%EB%A7%A4%EC%B6%9C%EC%95%A1_%EC%98%88%EC%B8%A1%7CPrivate_3%EC%9C%84(33675_98)_%7C_CatBoost_%7C_RandomizedSearchCV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GTSEO | Private 3위(33675.98) | CatBoost | RandomizedSearchCV
## 분석

https://dacon.io/competitions/official/235942/codeshare/5960?page=1&dtype=recent

In [1]:
!pip uninstall sklearn -y
!pip install --upgrade sklearn
!pip install scikit-learn==0.23.2 --user
# 설치 후 런타임 다시시작

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1310 sha256=ca5ecf4aa08bf9b680df0f367b2703e6ff0e59172e2ba3f70fb8d42a9a058adf
  Stored in directory: /root/.cache/pip/wheels/46/ef/c3/157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e
Successfully built sklearn
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.8 MB 8.3 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.4 requires scikit-learn>=1.0.0, but you have scikit-learn 0.23.2 which is incompatible.
imbalanced-learn 0.8.1 requires scikit-learn>=0.24, but you have scikit-learn 0.23.2 which is incompatible.


In [ ]:
import sklearn
sklearn.__version__

'0.23.2'

In [2]:
!pip install pycaret

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 320 kB 8.7 MB/s 
     |████████████████████████████████| 17.0 MB 66.7 MB/s 
     |████████████████████████████████| 1.7 MB 55.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 25.9 MB 84.0 MB/s 
     |████████████████████████████████| 262 kB 72.3 MB/s 
     |████████████████████████████████| 2.0 MB 57.5 MB/s 
     |████████████████████████████████| 1.3 MB 62.2 MB/s 
     |████████████████████████████████| 88 kB 9.6 MB/s 
     |████████████████████████████████| 636 kB 75.7 MB/s 
     |████████████████████████████████| 10.4 MB 49.5 MB/s 
     |████████████████████████████████| 167 kB 91.2 MB/s 
     |████████████████████████████████| 56 kB 6.7 MB/s 
     |████████████████████████████████

In [3]:
from pycaret.utils import enable_colab
enable_colab()

Colab mode enabled.


In [4]:
!pip install markupsafe==2.0.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 2.1.1
    Uninstalling MarkupSafe-2.1.1:
      Successfully uninstalled MarkupSafe-2.1.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas-profiling 3.2.0 requires markupsafe~=2.1.1, but you have markupsafe 2.0.1 which is incompatible.


In [1]:
import jinja2
from pycaret.regression import *
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 1.2 MB/s 


In [3]:
#함수
import numpy as np
import pandas as pd

#전처리
import datetime as dt

#결측치
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

#모델
import joblib
from catboost import CatBoostRegressor
from sklearn.model_selection import RandomizedSearchCV

#경고
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings(action='ignore', category=ConvergenceWarning) #경고메세지 끄기
warnings.simplefilter(action='ignore', category=FutureWarning) # FutureWarning 제거
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

#성능평가
from sklearn.metrics import mean_squared_error

def RMSE(y_pred,y):
  return mean_squared_error(y_pred,y)**0.5

#데이터
train=pd.read_csv('/content/train.csv')
test=pd.read_csv('/content/test.csv')
sample=pd.read_csv('/content/sample_submission.csv')

In [4]:
display(train.head())
print()
display(test.head())
print()

,id,Store,Date,Temperature,Fuel_Price,Promotion1,Promotion2,Promotion3,Promotion4,Promotion5,Unemployment,IsHoliday,Weekly_Sales
0,1,1,05/02/2010,42.31,2.572,NaN,NaN,NaN,NaN,NaN,8.106,False,1643690.90
1,2,1,12/02/2010,38.51,2.548,NaN,NaN,NaN,NaN,NaN,8.106,True,1641957.44
2,3,1,19/02/2010,39.93,2.514,NaN,NaN,NaN,NaN,NaN,8.106,False,1611968.17
3,4,1,26/02/2010,46.63,2.561,NaN,NaN,NaN,NaN,NaN,8.106,False,1409727.59
4,5,1,05/03/2010,46.50,2.625,NaN,NaN,NaN,NaN,NaN,8.106,False,1554806.68


,id,Store,Date,Temperature,Fuel_Price,Promotion1,Promotion2,Promotion3,Promotion4,Promotion5,Unemployment,IsHoliday
0,1,1,05/10/2012,68.55,3.617,8077.89,NaN,18.22,3617.43,3626.14,6.573,False
1,2,1,12/10/2012,62.99,3.601,2086.18,NaN,8.11,602.36,5926.45,6.573,False
2,3,1,19/10/2012,67.97,3.594,950.33,NaN,4.93,80.25,2312.85,6.573,False
3,4,1,26/10/2012,69.16,3.506,2585.85,31.75,6.00,1057.16,1305.01,6.573,False
4,5,2,05/10/2012,70.27,3.617,6037.76,NaN,10.04,3027.37,3853.40,6.170,False


## EDA

In [ ]:
train

## 데이터 전처리

In [5]:
#날짜 전처리
def date_conv(df):
  df['Date']=pd.to_datetime(df['Date'],format="%d/%m/%Y")
  df['Year']=df['Date'].dt.year
  df['Month']=df['Date'].dt.month
  df['Day']=df['Date'].dt.day
  df['Week']=df['Date'].dt.isocalendar().week
  df=df.drop(columns=['Date'])
  return df

train=date_conv(train)
test=date_conv(test)
train.shape

(6255, 16)

In [6]:
#isholiday 변수의 타입 변환(bool->int)
mapping={True:1,False:0}

train['IsHoliday']=train['IsHoliday'].astype('object')
test['IsHoliday']=test['IsHoliday'].astype('object')

train['IsHoliday']=train['IsHoliday'].map(mapping)
test['IsHoliday']=test['IsHoliday'].map(mapping)

train.shape

(6255, 16)

## 데이터 결측치

In [7]:
train_copy=train.copy()
test_copy=test.copy()

def null(df):
  df=IterativeImputer(random_state=2021).fit_transform(df)
  df=pd.DataFrame(df)
  if df.shape[1]==train_copy.shape[1]:
    df.columns=train_copy.columns
  else:
    df.columns=test_copy.columns
  return df

train=null(train)
test=null(test)
train.shape

(6255, 16)

In [8]:
#int형 전환(float->int)

def change_int(df):
  null(df)
  df['Store']=df['Store'].apply(lambda x:int(x))
  df['Year']=df['Year'].apply(lambda x:int(x))
  df['Month']=df['Month'].apply(lambda x:int(x))
  df['Day']=df['Day'].apply(lambda x:int(x))
  df['Week']=df['Week'].apply(lambda x:int(x))
  return df

train=change_int(train)
test=change_int(test)
train

,id,Store,Temperature,Fuel_Price,Promotion1,Promotion2,Promotion3,Promotion4,Promotion5,Unemployment,IsHoliday,Weekly_Sales,Year,Month,Day,Week
0,1.0,1,42.31,2.572,10187.176756,4080.964301,2998.807907,4638.248813,6199.244732,8.106,0.0,1643690.90,2010,2,5,5
1,2.0,1,38.51,2.548,10179.649970,4078.265127,2993.189216,4633.968812,6193.943659,8.106,1.0,1641957.44,2010,2,12,6
2,3.0,1,39.93,2.514,10077.798248,4031.822685,2888.161209,4576.087318,6102.557866,8.106,0.0,1611968.17,2010,2,19,7
3,4.0,1,46.63,2.561,9400.329944,3718.698348,2177.621841,4192.943359,5486.450234,8.106,0.0,1409727.59,2010,2,26,8
4,5.0,1,46.50,2.625,9888.124397,3943.320154,2686.956948,4468.823642,5928.534937,8.106,0.0,1554806.68,2010,3,5,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6250,6251.0,45,75.09,3.867,23641.300000,6.000000,92.930000,6988.310000,3992.130000,8.684,0.0,734297.87,2012,8,31,35
6251,6252.0,45,75.70,3.911,11024.450000,12.800000,52.630000,1854.770000,2055.700000,8.684,1.0,766512.66,2012,9,7,36
6252,6253.0,45,67.87,3.948,11407.950000,2568.572836,4.300000,3421.720000,5268.920000,8.684,0.0,702238.27,2012,9,14,37
6253,6254.0,45,65.32,4.038,8452.200000,92.280000,63.240000,2376.380000,8670.400000,8.684,0.0,723086.20,2012,9,21,38


In [13]:
train['Month'].value_counts()

4     630
7     630
3     585
6     585
8     585
9     585
2     540
5     540
12    450
10    405
11    360
1     360
Name: Month, dtype: int64

In [14]:
train['Year'].value_counts()

2011    2340
2010    2160
2012    1755
Name: Year, dtype: int64

In [15]:
test['Month'].value_counts()

10    180
Name: Month, dtype: int64

In [16]:
train['Year'].value_counts()

2011    2340
2010    2160
2012    1755
Name: Year, dtype: int64

## 모델링

In [10]:
cat=CatBoostRegressor()

#매개변수
params={'n_estimators':[200,500,1000,2000],
        'learning_rate':[0.1,0.05,0.01],
        'max_depth':[5,6,7,8],
        'subsample':[0.6,0.8,0.9,1.0]}

#예측 시 사용할 변수들
features=['Year','Month','Day','IsHoliday']

#모델, 성능(error) 저장
models=[]
models.append(0)
error=[]

#store의 개수가 45개였다. 
for store in range(1,46):
  train_store=train[train['Store']==store]
  #스토어 별로 모델을 돌리기 위함. 

  #연도별 데이터 분해
  train2010=train_store[(train_store['Year']==2010)&(train_store['Month']<=9)]
  train2011=train_store[(train_store['Year']==2011)&(train_store['Month']<=9)]
  train2012=train_store[(train_store['Year']==2012)&(train_store['Month']<9)]
  #test 데이터에서 예측할 것은 10월이므로 9월까지만의 데이터를 조정하는 것 같다. 

  #3가지(2010+2012,2011+2012,2010+2011+2012) 데이터 프레임 생성. 
  train0=pd.concat([train2010,train2012])
  train1=pd.concat([train2011,train2012])
  train2=pd.concat([train2010,train2011,train2012])
  #왜 이렇게 하는 걸까. 

  #학습
  model0=cat
  model0.fit(train0[features],train0['Weekly_Sales'])

  model1=cat
  model1.fit(train1[features],train1['Weekly_Sales'])

  model2=cat
  model2.fit(train2[features],train2['Weekly_Sales'])

  #예측
  y_val=train_store[(train_store['Year']==2012)&(train_store['Month']==9)]['Weekly_Sales']
  X_val=train_store[(train_store['Year']==2012)&(train_store['Month']==9)][features]
  #9로 성능을 테스트해보기 위함. 

  pred0=model0.predict(X_val)
  pred1=model1.predict(X_val)
  pred2=model2.predict(X_val)

  #RMSE로 예측 평가
  rmse0=RMSE(pred0,y_val)
  rmse1=RMSE(pred1,y_val)
  rmse2=RMSE(pred2,y_val)

  #3가지 RMSE 중 최소값을 이용해 데이터프레임 추출
  if rmse0<rmse1 and rmse0<rmse2:#rmse0이 최소 일때
    error.append(rmse0)
    train_part=train_store[(train_store['Year']==2010)&(train_store['Month']<=10)]
  elif rmse1<rmse2: #rmse1이 최소 일때
    error.append(rmse1)
    train_part=train_store[(train_store['Year']==2011)&(train_store['Month']<=10)]
  else:# rmse2가 최소 일때
    error.append(rmse2)
    train_part=pd.concat([train_store[(train_store['Year']==2010)&(train_store['Month']<=10)],
                          train_store[(train_store['Year']==2011)&(train_store['Month']<=10)]])
      #3가지(2010+2012,2011+2012,2010+2011+2012) 데이터 프레임 생성. 여기서 했던 거 처럼. 
      #rmse0은 train2010만, rmse1은 train2011만 rmse2는 train2010,2011 둘다 제공함

  #최종 학습 데이터
  train_final=pd.concat([train_part,train_store[train_store['Year']==2012]])
  X_train=train_final[features]
  y_train=train_final['Weekly_Sales']

  #학습 및 저장
  model=RandomizedSearchCV(cat,param_distributions=params,n_iter=30,cv=5,n_jobs=-1)
  model.fit(X_train,y_train)
  models.append(model)
  joblib.dump(models[store],open('store{0}.model'.format(store),'wb'))

print('평균 rmse :',np.mean(error))







스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
4:	learn: 19840.4468235	total: 757us	remaining: 151ms
5:	learn: 19513.6895960	total: 825us	remaining: 137ms
6:	learn: 19199.9636649	total: 977us	remaining: 139ms
7:	learn: 18894.7579906	total: 1.18ms	remaining: 147ms
8:	learn: 18548.9891800	total: 1.34ms	remaining: 147ms
9:	learn: 18240.8420147	total: 1.54ms	remaining: 152ms
10:	learn: 17977.4114098	total: 1.73ms	remaining: 155ms
11:	learn: 17755.1271799	total: 1.97ms	remaining: 162ms
12:	learn: 17471.8669411	total: 2.07ms	remaining: 157ms
13:	learn: 17226.8751330	total: 2.2ms	remaining: 155ms
14:	learn: 16976.6276443	total: 2.26ms	remaining: 149ms
15:	learn: 16709.5018894	total: 4.67ms	remaining: 287ms
16:	learn: 16477.0956396	total: 4.75ms	remaining: 275ms
17:	learn: 16257.9034641	total: 4.84ms	remaining: 264ms
18:	learn: 16047.4903390	total: 4.92ms	remaining: 254ms
19:	learn: 15813.0202788	total: 5.01ms	remaining: 245ms
20:	learn: 15599.0995048	total: 5.1ms	remaining: 238ms
21:	learn: 15350.041944

평균 rmse : 37028.95670182583

## 예측

In [11]:
pred=[]

for store in range(1,46):
  test_store=test[test['Store']==store]
  y=models[store].predict(test_store[features])
  pred+=list(y)

sample['Weekly_Sales']=pred
sample.head()

,id,Weekly_Sales
0,1,1.683971e+06
1,2,1.562556e+06
2,3,1.552222e+06
3,4,1.482853e+06
4,5,1.931857e+06
